In [50]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import numpy as np
import pandas as pd

[fast ai](https://forums.fast.ai/t/class-notes-section-2-wk2lec2/6818)

[kaggle home](https://www.kaggle.com/c/bluebook-for-bulldozers/data)

In [79]:
import re

def add_datepart(df, fldname):
    fld = df[fldname]
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)

In [80]:
dataset = pd.read_csv('./data/Train.csv', low_memory = False, parse_dates=["saledate"])
dataset.SalePrice = np.log(dataset.SalePrice)
obj_cols = dataset.select_dtypes(include = ['object']).columns
dataset.loc[:, obj_cols] = dataset.loc[:, obj_cols].fillna("NaN0")
add_datepart(dataset, "saledate")

In [81]:
validset = pd.read_csv('./data/Valid.csv', low_memory = False, parse_dates=["saledate"])
obj_cols = validset.select_dtypes(include = ['object']).columns
validset.loc[:, obj_cols] = validset.loc[:, obj_cols].fillna("NaN0")
add_datepart(validset, "saledate")

In [82]:
solnset = pd.read_csv('./data/ValidSolution.csv', low_memory = False)
solnset.SalePrice = np.log(solnset.SalePrice)
obj_cols = solnset.select_dtypes(include = ['object']).columns
solnset.loc[:, obj_cols] = solnset.loc[:, obj_cols].fillna("NaN0")

In [83]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
def encode_cols(df, dp = True):
    for header in df.columns:
        col = df[header]
        if col.dtype == "object":
            enc_col = le.fit_transform(col)
            df[header] = enc_col
encode_cols(dataset)
encode_cols(validset)
dataset.fillna(dataset.mean(), inplace = True)
validset.fillna(testset.mean(), inplace = True)

In [84]:
from sklearn.ensemble import RandomForestRegressor

In [85]:
%%time
y_train = dataset["SalePrice"]
X_train = dataset[[x for x in dataset.columns if x != "SalePrice"]]
rf = RandomForestRegressor(n_jobs = -1)
rf.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1min 53s, sys: 1.99 s, total: 1min 55s
Wall time: 39.6 s


In [86]:
from sklearn.metrics import mean_squared_log_error
from math import sqrt
predictions = rf.predict(validset)
print(predictions)
# out = sorted(zip(validset[validset.columns[0]], predictions), key = lambda x: x[0])
# print(list(out))
# print(solnset["SalePrice"])
print(sqrt(mean_squared_log_error(predictions, solnset["SalePrice"])))

[10.71052361 11.35637564 11.07446575 ...  9.95453745 10.00318754
 10.28407384]
0.0504775001218115
